In [ ]:
# Mount your google drive in google colab
from google.colab import drive
drive.mount('/content/drive')

!ls

Mounted at /content/drive
drive  sample_data


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import matplotlib.pyplot as plt


In [ ]:
#Methylation (Gene level, HM450K)
file_path = 'drive/MyDrive/Methylation__Meth450_csv_fillna.csv'
df = pd.read_csv(file_path)
df

,attrib_name,RBL2,VDAC3,ACTN1,ATP2A1,SFRP1,NIPA2,MAN1B1,TSEN34,LRRC16A,...,BAGE5,BAGE,BAGE2,BAGE4,BAGE3,MIR637,LOC100130932,GOLGA8F,GOLGA8G,MIR7-1
0,TCGA.3C.AAAU,-0.4444,-0.4548,0.2310,0.3568,-0.4737,-0.4817,0.4142,-0.4366,-0.4610,...,-0.0420,-0.0420,-0.0420,-0.0420,-0.0420,0.3420,0.2383,-0.1168,-0.1168,-0.344497
1,TCGA.3C.AALI,-0.4313,-0.4692,0.2859,0.3972,-0.2806,-0.4740,0.4373,-0.4527,-0.4643,...,-0.0640,-0.0640,-0.0640,-0.0640,-0.0640,0.3857,0.3783,-0.1271,-0.1271,-0.344497
2,TCGA.3C.AALJ,-0.4285,-0.4697,0.2664,0.3482,-0.0637,-0.4783,0.3738,-0.4430,-0.4416,...,-0.0873,-0.0873,-0.0873,-0.0873,-0.0873,0.3319,0.1929,0.0066,0.0066,-0.344497
3,TCGA.3C.AALK,-0.4043,-0.4714,0.2174,0.3533,-0.0440,-0.4827,0.3704,-0.4430,-0.4558,...,0.1426,0.1426,0.1426,0.1426,0.1426,0.3457,0.2445,-0.0071,-0.0071,-0.344497
4,TCGA.4H.AAAK,-0.4083,-0.4651,0.1717,0.3406,-0.0880,-0.4788,0.4143,-0.4442,-0.4675,...,0.0218,0.0218,0.0218,0.0218,0.0218,0.3542,0.3128,-0.0929,-0.0929,-0.344497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,TCGA.WT.AB44,-0.3903,-0.4683,0.2731,0.3682,-0.1018,-0.4842,0.3842,-0.4408,-0.4601,...,0.2111,0.2111,0.2111,0.2111,0.2111,0.3322,0.2746,-0.0284,-0.0284,-0.344497
779,TCGA.XX.A899,-0.4500,-0.4721,0.2597,0.4152,0.0244,-0.4843,0.4211,-0.4391,-0.4622,...,0.1998,0.1998,0.1998,0.1998,0.1998,0.3908,0.4161,-0.0702,-0.0702,-0.259800
780,TCGA.XX.A89A,-0.4237,-0.4649,0.1951,0.3586,-0.4071,-0.4862,0.4494,-0.4609,-0.4639,...,0.1236,0.1236,0.1236,0.1236,0.1236,0.4054,0.4089,-0.1210,-0.1210,-0.247000
781,TCGA.Z7.A8R5,-0.4197,-0.4688,0.1128,0.3803,-0.1014,-0.4843,0.4244,-0.4283,-0.4676,...,0.2220,0.2220,0.2220,0.2220,0.2220,0.3571,0.2107,0.0495,0.0495,-0.344497


In [ ]:
#Separate text descriptions from numeric attributes
text_descriptions = df.iloc[:,0]
numeric_attributes = df.iloc[:,1:]
print(numeric_attributes)
print(numeric_attributes.shape[1])

       RBL2   VDAC3   ACTN1  ATP2A1   SFRP1   NIPA2  MAN1B1  TSEN34  LRRC16A  \
0   -0.4444 -0.4548  0.2310  0.3568 -0.4737 -0.4817  0.4142 -0.4366  -0.4610   
1   -0.4313 -0.4692  0.2859  0.3972 -0.2806 -0.4740  0.4373 -0.4527  -0.4643   
2   -0.4285 -0.4697  0.2664  0.3482 -0.0637 -0.4783  0.3738 -0.4430  -0.4416   
3   -0.4043 -0.4714  0.2174  0.3533 -0.0440 -0.4827  0.3704 -0.4430  -0.4558   
4   -0.4083 -0.4651  0.1717  0.3406 -0.0880 -0.4788  0.4143 -0.4442  -0.4675   
..      ...     ...     ...     ...     ...     ...     ...     ...      ...   
778 -0.3903 -0.4683  0.2731  0.3682 -0.1018 -0.4842  0.3842 -0.4408  -0.4601   
779 -0.4500 -0.4721  0.2597  0.4152  0.0244 -0.4843  0.4211 -0.4391  -0.4622   
780 -0.4237 -0.4649  0.1951  0.3586 -0.4071 -0.4862  0.4494 -0.4609  -0.4639   
781 -0.4197 -0.4688  0.1128  0.3803 -0.1014 -0.4843  0.4244 -0.4283  -0.4676   
782 -0.4682 -0.4576  0.3352  0.3621 -0.4837 -0.4846  0.4343 -0.4688  -0.4747   

       CNBP  ...   BAGE5    BAGE   BAGE

In [ ]:
pip install keras-tuner --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 23.0 MB/s eta 0:00:00


In [ ]:
#https://www.analyticsvidhya.com/blog/2021/06/dimensionality-reduction-using-autoencoders-in-python/
import math
import pandas as pd
import tensorflow as tf
import kerastuner.tuners as kt
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
import numpy as np


# Train the autoencoder with the training data.
class AutoEncoders(Model):

  def __init__(self, output_units):

    super().__init__()
    self.encoder = Sequential(
        [
          Dense(512, activation="relu"),#
          Dense(256, activation="relu"),
          Dense(128, activation="relu"),

        ]
    )

    self.decoder = Sequential(
        [
          Dense(256, activation="relu"),
          Dense(512, activation="relu"),
          Dense(output_units, activation="sigmoid")
        ]
    )

  def call(self, inputs):

    encoded = self.encoder(inputs)
    decoded = self.decoder(encoded)
    return decoded


def create_and_train_autoencoder(x_train_scaled,x_test_scaled):
    auto_encoder = AutoEncoders(len(x_train_scaled.columns))
    auto_encoder.compile(
        loss='mae',
        metrics=['mae'],
        #optimizer='adam'
        optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# 0.001 to avoid overlearning
    )
    history = auto_encoder.fit(
        x_train_scaled,
        x_train_scaled,
        epochs=50,
        batch_size=32,
        validation_data=(x_test_scaled, x_test_scaled),
        verbose=0
    )
    return history




Using TensorFlow backend


<ipython-input-6-3123b568cb15>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner.tuners as kt


In [ ]:
# Split the data
standard_scaler = MinMaxScaler()
numeric_attributes_scaled = pd.DataFrame(standard_scaler.fit_transform(numeric_attributes),columns=numeric_attributes.columns)

In [ ]:
x_train_scaled, x_test_scaled = train_test_split(numeric_attributes_scaled, test_size=0.3, random_state=42)

auto_encoder = AutoEncoders(len(x_train_scaled.columns))
auto_encoder.compile(
    loss='mae',
    metrics=['mae'],
    #optimizer='adam'
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)# 0.001 to avoid overlearning
)
history = auto_encoder.fit(
    x_train_scaled,
    x_train_scaled,
    epochs=50,
    batch_size=32,
    validation_data=(x_test_scaled, x_test_scaled),
    #verbose=0
)
encoder_layer = auto_encoder.layers[0]  # Get the encoder layer
X_encoded = pd.DataFrame(encoder_layer.predict(numeric_attributes_scaled))  # Reduced_df
#X_encoded

Epoch 1/50
18/18 [==============================] - 15s 775ms/step - loss: 0.1572 - mae: 0.1572 - val_loss: 0.1161 - val_mae: 0.1161
Epoch 2/50
18/18 [==============================] - 13s 755ms/step - loss: 0.1132 - mae: 0.1132 - val_loss: 0.1146 - val_mae: 0.1146
Epoch 3/50
18/18 [==============================] - 11s 609ms/step - loss: 0.1118 - mae: 0.1118 - val_loss: 0.1117 - val_mae: 0.1117
Epoch 4/50
18/18 [==============================] - 7s 361ms/step - loss: 0.1093 - mae: 0.1093 - val_loss: 0.1040 - val_mae: 0.1040
Epoch 5/50
18/18 [==============================] - 8s 466ms/step - loss: 0.1028 - mae: 0.1028 - val_loss: 0.1015 - val_mae: 0.1015
Epoch 6/50
18/18 [==============================] - 7s 374ms/step - loss: 0.1003 - mae: 0.1003 - val_loss: 0.0983 - val_mae: 0.0983
Epoch 7/50
18/18 [==============================] - 8s 458ms/step - loss: 0.0975 - mae: 0.0975 - val_loss: 0.0963 - val_mae: 0.0963
Epoch 8/50
18/18 [==============================] - 7s 390ms/step - loss:

In [ ]:
#Epoch 50/50
#18/18 [==============================] - 7s 398ms/step - loss: 0.0691 - mae: 0.0691 - val_loss: 0.0833 - val_mae: 0.0833
#25/25 [==============================] - 1s 47ms/step

In [ ]:
X_encoded

,0,1,2,3,4,5,6,7,8,9,...,118,119,120,121,122,123,124,125,126,127
0,14.625108,0.000000,0.0,8.790771,0.0,0.0,0.0,0.0,10.209114,0.0,...,0.000000,0.000000,0.0,17.935270,0.000000,7.199439,6.432836,0.0,0.0,0.746716
1,15.384162,2.610127,0.0,6.576795,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,5.228476,0.000000,9.667120,6.921007,0.0,0.0,4.977176
2,12.510753,0.000000,0.0,4.793648,0.0,0.0,0.0,0.0,0.505122,0.0,...,0.000000,4.193615,0.0,7.433645,3.338945,11.594288,6.240092,0.0,0.0,1.348448
3,19.158442,0.000000,0.0,4.066125,0.0,0.0,0.0,0.0,2.295568,0.0,...,1.128648,7.294955,0.0,5.690634,13.832063,12.634056,5.871393,0.0,0.0,0.688025
4,21.311453,0.000000,0.0,7.159411,0.0,0.0,0.0,0.0,6.108991,0.0,...,0.329032,0.000000,0.0,8.225735,9.221536,9.559469,7.731924,0.0,0.0,1.033152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,26.277332,0.000000,0.0,5.629574,0.0,0.0,0.0,0.0,4.339798,0.0,...,2.593006,1.302498,0.0,8.831540,14.045652,12.590933,9.586295,0.0,0.0,0.000000
779,15.551846,6.309134,0.0,3.641717,0.0,0.0,0.0,0.0,6.995820,0.0,...,0.000000,10.256507,0.0,1.200140,16.881884,7.373714,6.656590,0.0,0.0,3.595672
780,9.584288,8.129096,0.0,6.884301,0.0,0.0,0.0,0.0,7.602070,0.0,...,0.000000,0.000000,0.0,1.877149,1.355946,1.790250,4.644177,0.0,0.0,6.035753
781,19.851770,1.451877,0.0,6.806647,0.0,0.0,0.0,0.0,4.852437,0.0,...,0.515243,2.663803,0.0,5.514261,14.103909,7.286576,3.491872,0.0,0.0,0.000000


In [ ]:
text_descriptions

0      TCGA.3C.AAAU
1      TCGA.3C.AALI
2      TCGA.3C.AALJ
3      TCGA.3C.AALK
4      TCGA.4H.AAAK
           ...     
778    TCGA.WT.AB44
779    TCGA.XX.A899
780    TCGA.XX.A89A
781    TCGA.Z7.A8R5
782    TCGA.Z7.A8R6
Name: attrib_name, Length: 783, dtype: object

In [ ]:
X_encoded.insert(0, 'attrib_name', text_descriptions)
X_encoded

,attrib_name,0,1,2,3,4,5,6,7,8,...,118,119,120,121,122,123,124,125,126,127
0,TCGA.3C.AAAU,14.625108,0.000000,0.0,8.790771,0.0,0.0,0.0,0.0,10.209114,...,0.000000,0.000000,0.0,17.935270,0.000000,7.199439,6.432836,0.0,0.0,0.746716
1,TCGA.3C.AALI,15.384162,2.610127,0.0,6.576795,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.0,5.228476,0.000000,9.667120,6.921007,0.0,0.0,4.977176
2,TCGA.3C.AALJ,12.510753,0.000000,0.0,4.793648,0.0,0.0,0.0,0.0,0.505122,...,0.000000,4.193615,0.0,7.433645,3.338945,11.594288,6.240092,0.0,0.0,1.348448
3,TCGA.3C.AALK,19.158442,0.000000,0.0,4.066125,0.0,0.0,0.0,0.0,2.295568,...,1.128648,7.294955,0.0,5.690634,13.832063,12.634056,5.871393,0.0,0.0,0.688025
4,TCGA.4H.AAAK,21.311453,0.000000,0.0,7.159411,0.0,0.0,0.0,0.0,6.108991,...,0.329032,0.000000,0.0,8.225735,9.221536,9.559469,7.731924,0.0,0.0,1.033152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
778,TCGA.WT.AB44,26.277332,0.000000,0.0,5.629574,0.0,0.0,0.0,0.0,4.339798,...,2.593006,1.302498,0.0,8.831540,14.045652,12.590933,9.586295,0.0,0.0,0.000000
779,TCGA.XX.A899,15.551846,6.309134,0.0,3.641717,0.0,0.0,0.0,0.0,6.995820,...,0.000000,10.256507,0.0,1.200140,16.881884,7.373714,6.656590,0.0,0.0,3.595672
780,TCGA.XX.A89A,9.584288,8.129096,0.0,6.884301,0.0,0.0,0.0,0.0,7.602070,...,0.000000,0.000000,0.0,1.877149,1.355946,1.790250,4.644177,0.0,0.0,6.035753
781,TCGA.Z7.A8R5,19.851770,1.451877,0.0,6.806647,0.0,0.0,0.0,0.0,4.852437,...,0.515243,2.663803,0.0,5.514261,14.103909,7.286576,3.491872,0.0,0.0,0.000000


In [ ]:
X_encoded.to_csv("drive/MyDrive/Methylation__Meth450_128.csv", index=False)